## Vectorize Python Client Full Tutorial


In [3]:
!pip install vectorize-client --upgrade

In [6]:
import getpass
org=input("Vectorize Organization ID:")
token=getpass.getpass("Vectorize Token:")

Vectorize Organization ID:582893ba-291c-4ec7-a6ee-e85c26888817
Vectorize Token:··········


In [13]:
import vectorize_client as v


api = v.ApiClient(v.Configuration(access_token=token))
pipelines = v.PipelinesApi(api)

response = pipelines.get_pipelines(org)
for pipeline in response.data:
    print("Pipeline: " + pipeline.name)

Pipeline: My Pipeline From API
Pipeline: from api
Pipeline: Test pipeline
Pipeline: Test pipeline


In [25]:
!wget https://www.apple.com/newsroom/pdfs/fy2024-q1/FY24_Q1_Consolidated_Financial_Statements.pdf --output-file apple.pdf

In [8]:
connectors_api = v.ConnectorsApi(api)
response = connectors_api.create_source_connector(org, [{
        "type": "FILE_UPLOAD",
        "name": "From API"
    }])
source_connector_id = response.connectors[0].id
source_connector_id

'cdfa4981-c0fa-4b1d-9fba-83845cc103f0'

In [10]:
import urllib3, json, os
http = urllib3.PoolManager()
file_path = "apple.pdf"

uploads_api = v.UploadsApi(api)

metadata = {"created-from-api": True}

upload_response = uploads_api.start_file_upload_to_connector(
        org, source_connector_id, v.StartFileUploadToConnectorRequest(
            name=file_path.split("/")[-1],
            content_type="application/pdf",
            # add additional metadata that will be stored along with each chunk in the vector database
            metadata=json.dumps(metadata))
)

with open(file_path, "rb") as f:
    response = http.request("PUT", upload_response.upload_url, body=f, headers={"Content-Type": "application/pdf", "Content-Length": str(os.path.getsize(file_path))})
if response.status != 200:
    print("Upload failed: ", response.data)
else:
    print("Upload successful")

Upload successful


In [11]:
ai_platforms = connectors_api.get_ai_platform_connectors(org)
builtin_ai_platform = [c.id for c in ai_platforms.ai_platform_connectors if c.type == "VECTORIZE"][0]

vector_databases = connectors_api.get_destination_connectors(org)
builtin_vector_db = [c.id for c in vector_databases.destination_connectors if c.type == "VECTORIZE"][0]

In [12]:
response = pipelines.create_pipeline(org, v.PipelineConfigurationSchema(
    source_connectors=[v.SourceConnectorSchema(id=source_connector_id, type="FILE_UPLOAD", config={})],
    destination_connector=v.DestinationConnectorSchema(id=builtin_vector_db, type="VECTORIZE", config={}),
    ai_platform=v.AIPlatformSchema(id=builtin_ai_platform, type="VECTORIZE", config={}),
    pipeline_name="My Pipeline From API",
    schedule=v.ScheduleSchema(type="manual")
))
pipeline_id = response.data.id
pipeline_id

'aipf9ab5-702c-40de-a2f7-8a2139129ba5'

In [15]:
pipelines = v.PipelinesApi(api)
response = pipelines.retrieve_documents(org, pipeline_id, v.RetrieveDocumentsRequest(
    question="Apple RSU activity",
    num_results=5,
))
print(response)
for doc in response.documents:
  print(str(doc.relevancy) +" - " + doc.text)

question='Apple RSU activity' documents=[Document(relevancy=0.9959771, id='bd649160-a790-4b81-a4f4-4655ffe3a71f-87ac4bd677c3b84b4634c5da51427f20a2ab140e4b56b213cd3f4a0f1c356e27-84', text='2014 Employee Stock Plan\n\nThe Apple Inc. 2014 Employee Stock Plan, as amended and restated (the “2014 Plan”), is a shareholder-approved plan that\nprovided for broad-based equity grants to employees, including executive officers. The 2014 Plan permitted the granting of the\nsame types of equity awards with substantially the same terms as the 2022 Plan. The 2014 Plan also permitted the granting of\ncash bonus awards. In the third quarter of 2022, the Company terminated the authority to grant new awards under the 2014\nPlan.\n\nApple Inc. | 2024 Form 10-K | 44\n\n\nRestricted Stock Units\n\nA summary of the Company’s RSU activity and related information for 2024, 2023 and 2022, is as follows:\n\nNumber of\nRSUs\n\n(in thousands)\n\nWeighted-Average\nGrant-Date Fair\nValue Per RSU\n\nAggregate\nFair Va

In [17]:
pipelines_api = v.PipelinesApi(api)
response = pipelines.start_deep_research(org, pipeline_id, v.StartDeepResearchRequest(
        # make sure to include a relevant prompt here
        query="Generate a report on Apple RSE 2024",
        # optionally enable additional search on the web
        web_search=False,
))
research_id = response.research_id

In [ ]:
while True:
    response = pipelines.get_deep_research_result(org, pipeline_id, research_id)
    if response.ready:
        if response.data.success:
            print(response.data.markdown)
        else:
            print("Deep Research failed: ", response.data.error)
        break
    print("not ready")

In [22]:
from pathlib import Path
import urllib3, os

files_api = v.FilesApi(api)

content_type="application/pdf"
file_path="apple.pdf"

start_file_upload_response = files_api.start_file_upload(org, start_file_upload_request=v.StartFileUploadRequest(
    content_type=content_type,
    name="My file.pdf",
))

http = urllib3.PoolManager()

with open(file_path, "rb") as f:
    response = http.request("PUT", start_file_upload_response.upload_url, body=f, headers={"Content-Type": "application/pdf", "Content-Length": str(os.path.getsize(file_path))})
    if response.status != 200:
        print("Upload failed: ", response.data)
    else:
        print("Upload successful")

extraction_api = v.ExtractionApi(api)

response = extraction_api.start_extraction(org, start_extraction_request=v.StartExtractionRequest(
        file_id=start_file_upload_response.file_id
))
extraction_id = response.extraction_id

Upload successful


In [23]:
while True:
    response = extraction_api.get_extraction_result(org, extraction_id)
    if response.ready:
        if response.data.success:
            print(response.data.text)
        else:
            print("Extraction failed: ", response.data.error)
        break
    print("not ready")

not ready
not ready
not ready
not ready
not ready
not ready
not ready
not ready
not ready
not ready
not ready
# Financial Report of Apple Inc. for the Fiscal Year 2024

## A Comprehensive Analysis of Financial Performance, Risks, and Future Outlook

## Executive Summary

This report provides an in-depth analysis of Apple Inc.'s financial status for the fiscal year 2024. It summarizes key financial performance indicators, including revenue, profitability, cash flow, and significant financial risks. The report also examines market performance and future outlook, offering insights into the company's strategic initiatives and potential challenges.

## Table of Contents

1. Company Overview

* Company Background
* Key Business Segments
* Market Position

2. Financial Highlights

* Revenue Summary
* Profitability Metrics
* Cash Flow Overview

3. Balance Sheet Analysis

* Assets Breakdown
* Liabilities Overview
* Shareholders' Equity

4. Income Statement Analysis

* Revenue Sources
* Operatin